<a href="https://colab.research.google.com/github/Harish203201/Deep-Learning/blob/master/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tensorflow as tf
style_image_path = "index2.jpg"
content_image_path = "IMG_20200805_134456_278.jpg"

In [ ]:
def load_img(path_to_img, dimension):
  img = tf.io.read_file(path_to_img)
  img = tf.io.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = img[tf.newaxis, :]
  shape = tf.cast(tf.shape(img)[1:-1], tf.float32)
  short_dim = min(shape)
  scale = dimension / short_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  image = tf.image.resize(img, new_shape)
  image = tf.image.resize_with_crop_or_pad(image, dimension, dimension)
  return image

content_image = load_img(content_image_path, 384)
style_image = load_img(style_image_path, 256)

In [ ]:
style_predict_path = tf.keras.utils.get_file('style_predict.tflite', 'https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/prediction/1?lite-format=tflite')
style_transform_path = tf.keras.utils.get_file('style_transform.tflite', 'https://tfhub.dev/google/lite-model/magenta/arbitrary-image-stylization-v1-256/int8/transfer/1?lite-format=tflite')

In [ ]:
from keras.preprocessing.image import array_to_img
content_blending_ratio = 0.2 

def run_style_predict(preprocessed_style_image):
  interpreter = tf.lite.Interpreter(model_path=style_predict_path)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  interpreter.set_tensor(input_details[0]["index"], preprocessed_style_image)
  interpreter.invoke()
  style_bottleneck = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()
  return style_bottleneck

style_bottleneck = run_style_predict(style_image)

def run_style_transform(style_bottleneck, preprocessed_content_image):
  interpreter = tf.lite.Interpreter(model_path=style_transform_path)
  input_details = interpreter.get_input_details()
  interpreter.allocate_tensors()
  interpreter.set_tensor(input_details[0]["index"], preprocessed_content_image)
  interpreter.set_tensor(input_details[1]["index"], style_bottleneck)
  interpreter.invoke()
  stylized_image = interpreter.tensor(
      interpreter.get_output_details()[0]["index"]
      )()

  return stylized_image

style_bottleneck_content = run_style_predict(
    load_img(content_image_path, 256)
)
style_bottleneck_blended = content_blending_ratio * style_bottleneck_content + (1 - content_blending_ratio) * style_bottleneck
stylized_image = run_style_transform(style_bottleneck_blended, content_image)
array_to_img(stylized_image.squeeze(axis=0)).save("stylized_image.png")